In [1]:
# Imports
# Utils
import matplotlib as plt
import numpy as np
import wandb
# DL libraries
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader
# User libraries
from datasets.segmentationdataset import SegmentationDataset
from models.unet_model import UNet
from trainers.segmentation_model_trainer import train_segmentation_model
from util import config

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [2]:
# Get the data
train_segmentation_dataset = SegmentationDataset(
    config.SEGMENTATION_DATA_PATH_TRAIN_FEATURES,
    config.SEGMENTATION_DATA_PATH_TRAIN_LABELS,
    config.SEGMENTATION_TRAIN_TRANSFORMATIONS_BOTH
    )

test_segmentation_dataset = SegmentationDataset(
    config.SEGMENTATION_DATA_PATH_TEST_FEATURES,
    config.SEGMENTATION_DATA_PATH_TEST_LABELS
    )

# Place the datasets in dataloaders
train_segmentation_dataloader = DataLoader(train_segmentation_dataset, batch_size=config.SEGMENTATION_BATCH_SIZE)
test_segmentation_dataloader = DataLoader(test_segmentation_dataset, batch_size=1)



# Setup

In [3]:
# Get the model
model = UNet(n_channels=3, n_classes=1, bilinear=False)
model.to(config.DEVICE)

# Set the optimizer
optimizer = optim.Adam(model.parameters(), lr=config.SEGMENTATION_LR)

# Set the loss fn
criteria = nn.CrossEntropyLoss()

# Set the gradient scaler
grad_scaler = torch.cuda.amp.grad_scaler.GradScaler()


# Setup weights and biasses
#wandb.login()

# Start wandb
#wandb.init(
#    settings=wandb.Settings(start_method="fork"),
#    project="test-project", 
#    name=f"experiment_1", 
#    config={
#        "learning_rate": config.SEGMENTATION_LR,
#        "batch_size": config.SEGMENTATION_BATCH_SIZE,
#        "epochs": config.SEGMENTATION_EPOCHS,
#    }
#)

/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: robberdg. Use `wandb login --relogin` to force relogin
Thread HandlerThread:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/internal/internal_util.py", line 50, in run
    self._run()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/internal/internal_util.py", line 101, in _run
    self._process(record)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/internal/internal.py", line 263, in _process
    self._hm.handle(record)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/internal/handler.p

wandb: ERROR Internal wandb error: file data was not synced


# Training

In [8]:
for epoch in range(config.SEGMENTATION_EPOCHS):
    # Set the model in training mode
    model.train()

    # Train the model
    train_loss_this_epoch = train_segmentation_model(
        model,
        optimizer,
        criteria,
        grad_scaler,
        train_segmentation_dataloader
    )
    
    # Set the model in evaluation mode
    model.eval()

    # Validate the model
    val_loss_this_epoch, sample_image_array = validate_segmentation_model(
        model,
        criteria,
        test_segmentation_dataloader,
        test_segmentation_dataset
    )

    # convert the image array into an image
    sample_image = wandb.Image(sample_image_array, caption="Top: Output, Bottom: Input")

    # Log the train loss this epoch
    wandb.log({
        'train_loss': train_loss_this_epoch/len(train_segmentation_dataloader.dataset),
        'val_loss': val_loss_this_epoch/len(test_segmentation_dataloader.dataset),
        'sample_image': sample_image
    })


# Mark the run as finished
wandb.finish()




    

Error in callback <function _WandbInit._resume_backend at 0x7fac5cdb3250> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

  0%|          | 0/312 [00:00<?, ?it/s]


TypeError: Unexpected type <class 'numpy.ndarray'>

Error in callback <function _WandbInit._pause_backend at 0x7fac5cdb3400> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe